In [1]:
# 모듈 임포트
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

In [ ]:
# 상수 정의


In [ ]:
# 데이터셋 정의
class fiveDataset(Dataset):
    def __init__(self, ):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = self.data[idx]
        y = self.labels[idx]
        return x, y

In [ ]:
# 신경망 정의
class fiveClassifier(nn.Module):
    def __init__(self):
        super(fiveClassifier, self).__init__()
        self.fc1 = nn.Linear(3, 5)  # 입력: 3차원, 출력: 5차원
        self.fc2 = nn.Linear(5, 2)   # 입력: 5차원, 출력: 2차원 (이진 분류)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
dataset = fiveDataset([1, 2, 3, 4, 5], [0, 0, 1, 1, 0])
dataloader = DataLoader(dataset, batch_size=10, shuffle=True)